In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix

2025-11-09 13:39:48.099505: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762695588.526238      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762695588.644728      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [8]:
import matplotlib
import sklearn
import transformers

In [9]:
# Print versions
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)
print("seaborn version:", sns.__version__)
print("matplotlib version:", matplotlib.__version__)
print("nltk version:", nltk.__version__)
print("scikit-learn version:", sklearn.__version__)
print("transformers version:", transformers.__version__)
print("tensorflow version:", tf.__version__)

pandas version: 2.2.3
numpy version: 1.26.4
seaborn version: 0.12.2
matplotlib version: 3.7.2
nltk version: 3.9.2
scikit-learn version: 1.2.2
transformers version: 4.53.3
tensorflow version: 2.18.0


In [ ]:
data = pd.read_csv("/kaggle/input/mobile-reviews-sentiment-and-specification/Mobile Reviews Sentiment.csv")


In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data = data[["review_text","sentiment","battery_life_rating","camera_rating", "performance_rating", "design_rating", "display_rating"]].copy()

In [ ]:
data.info()

In [ ]:
data.duplicated().sum()

In [ ]:
data.drop_duplicates(inplace=True)
data.reset_index(inplace=True)

In [ ]:
data.info()

In [ ]:
columns = ["sentiment", "battery_life_rating", "camera_rating", "performance_rating", "design_rating", "display_rating"]
for col in columns:
    values = data[col].value_counts().sort_index()
    print(f"\nColumn: {col}")
    print(values)

In [ ]:
for col in columns:
    plt.figure(figsize=(6,4))
    sns.countplot(x=col, data=data, order=sorted(data[col].unique()))
    plt.title(f"Distribution of {col}")
    plt.show()


In [ ]:
data["review_text"][4]

In [ ]:
def clean_review(text):
    if not isinstance(text,str):
        return ""
    text = text.lower()
    text = re.sub(f'\s+',' ', text).strip()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\d+', '', text)

    return text

In [ ]:
data["clean_review"] = data["review_text"].apply(clean_review)

In [ ]:
le = LabelEncoder()
data['sentiment_label'] = le.fit_transform(data['sentiment'])

X = data['clean_review'].values

y_ratings = data[["battery_life_rating","camera_rating","performance_rating","design_rating","display_rating"]].values
y_sentiment = data['sentiment_label'].values

X_train, X_val, y_ratings_train, y_ratings_val, y_sentiment_train, y_sentiment_val = train_test_split(
    X, y_ratings, y_sentiment, test_size=0.25, random_state=42
)

In [ ]:
classes = np.array([0,1,2])
y_sentiment = data['sentiment_label'].values
class_weights = compute_class_weight('balanced', classes=classes, y=y_sentiment)
class_weight_dict = dict(zip(classes, class_weights))

print("\nClass Weights:")
print(class_weight_dict)

print("\nLabel Encoding:")
for i, class_label in enumerate(le.classes_):
    print(f"{class_label}: {i}")

In [ ]:
# Tokenize Text

MODEL_NAME = "bert-base-uncased"
MAX_LEN = 128

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

def tokenize_texts(texts, max_len=MAX_LEN):
    return tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=max_len,
        return_tensors="tf"
    )

train_encodings = tokenize_texts(X_train)
val_encodings = tokenize_texts(X_val)


In [ ]:
# Convert to plain dict for tf.data
train_encodings_tf = {
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask']
}
val_encodings_tf = {
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask']
}


In [ ]:
# Create TensorFlow Datasets

val_fraction = 0.1
train_size = int((1 - val_fraction) * len(X_train))
val_size = len(X_train) - train_size

train_dataset = tf.data.Dataset.from_tensor_slices((
    train_encodings_tf,
    {
        "battery": y_ratings_train[:, 0],
        "camera": y_ratings_train[:, 1],
        "performance": y_ratings_train[:, 2],
        "design": y_ratings_train[:, 3],
        "display": y_ratings_train[:, 4],
        "sentiment": y_sentiment_train
    }
)).shuffle(1000)

# Split datasets
train_dataset_final = train_dataset.take(train_size).batch(16).prefetch(tf.data.AUTOTUNE)
val_dataset_small = train_dataset.skip(train_size).batch(16).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((
    val_encodings_tf,
    {
        "battery": y_ratings_val[:, 0],
        "camera": y_ratings_val[:, 1],
        "performance": y_ratings_val[:, 2],
        "design": y_ratings_val[:, 3],
        "display": y_ratings_val[:, 4],
        "sentiment": y_sentiment_val
    }
)).batch(16).prefetch(tf.data.AUTOTUNE)

In [ ]:
# # Build Multi-Task BERT Model

# bert_model = TFBertModel.from_pretrained(MODEL_NAME)

# # Input layers
# input_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
# attention_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")

# # Wrap BERT in Lambda to accept KerasTensors
# pooled_output = tf.keras.layers.Lambda(
#     lambda x: bert_model(input_ids=x[0], attention_mask=x[1])[1],
#     output_shape=(768,)
# )([input_ids, attention_mask])

# # Aspect rating heads (regression)
# battery = tf.keras.layers.Dense(1, name="battery")(pooled_output)
# camera = tf.keras.layers.Dense(1, name="camera")(pooled_output)
# performance = tf.keras.layers.Dense(1, name="performance")(pooled_output)
# design = tf.keras.layers.Dense(1, name="design")(pooled_output)
# display = tf.keras.layers.Dense(1, name="display")(pooled_output)

# # Sentiment head (classification)
# sentiment = tf.keras.layers.Dense(3, activation='softmax', name="sentiment")(pooled_output)

# # Create model
# model = tf.keras.Model(
#     inputs=[input_ids, attention_mask],
#     outputs=[battery, camera, performance, design, display, sentiment]
# )


# # Convert to tensor
# class_weight_tensor = tf.constant([class_weight_dict[0],
#                                    class_weight_dict[1],
#                                    class_weight_dict[2]], dtype=tf.float32)


In [ ]:
@tf.keras.utils.register_keras_serializable()
class BertPoolingLayer(tf.keras.layers.Layer):
    def __init__(self, model_name="bert-base-uncased", trainable=True, **kwargs):
        super(BertPoolingLayer, self).__init__(**kwargs)
        self.bert_model = TFBertModel.from_pretrained(model_name)
        self.bert_model.trainable = trainable

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.pooler_output  # pooled_output

    def get_config(self):
        config = super(BertPoolingLayer, self).get_config()
        config.update({
            "model_name": self.bert_model.name,
            "trainable": self.bert_model.trainable
        })
        return config

In [ ]:
# Inputs
input_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")

# Use custom layer
pooled_output = BertPoolingLayer(MODEL_NAME)([input_ids, attention_mask])

# Regression heads
battery = tf.keras.layers.Dense(1, name="battery")(pooled_output)
camera = tf.keras.layers.Dense(1, name="camera")(pooled_output)
performance = tf.keras.layers.Dense(1, name="performance")(pooled_output)
design = tf.keras.layers.Dense(1, name="design")(pooled_output)
display = tf.keras.layers.Dense(1, name="display")(pooled_output)

# Sentiment classification
sentiment = tf.keras.layers.Dense(3, activation='softmax', name="sentiment")(pooled_output)

# Build model
model = tf.keras.Model(
    inputs=[input_ids, attention_mask],
    outputs=[battery, camera, performance, design, display, sentiment]
)


In [ ]:
# @tf.keras.utils.register_keras_serializable()
# def weighted_sentiment_loss(y_true, y_pred):
#     y_true_int = tf.cast(y_true, tf.int32)
#     weights = tf.gather(class_weight_tensor, y_true_int)
#     unweighted_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
#     weighted_loss = unweighted_loss * weights
#     return tf.reduce_mean(weighted_loss)

In [ ]:
CLASS_WEIGHTS = tf.constant([2.5679162609542354, 1.2814625850340136, 0.5463821411923137], dtype=tf.float32)

@tf.keras.utils.register_keras_serializable()
def weighted_sentiment_loss(y_true, y_pred):
    y_true_int = tf.cast(y_true, tf.int32)
    weights = tf.gather(CLASS_WEIGHTS, y_true_int)
    unweighted_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    weighted_loss = unweighted_loss * weights
    return tf.reduce_mean(weighted_loss)


In [ ]:
# Compile model with loss_weights to prioritize sentiment
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss={
        "battery": "mse",
        "camera": "mse",
        "performance": "mse",
        "design": "mse",
        "display": "mse",
        "sentiment": weighted_sentiment_loss
    },
    loss_weights={
        "battery": 0.15,
        "camera": 0.15,
        "performance": 0.15,
        "design": 0.15,
        "display": 0.15,
        "sentiment": 1.5
    },
    metrics={
        "battery": "mae",
        "camera": "mae",
        "performance": "mae",
        "design": "mae",
        "display": "mae",
        "sentiment": "accuracy"
    }
)

model.summary()

In [ ]:
# Add early stopping and train for more epoch
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=2,
    restore_best_weights=True
)

# Training 
history = model.fit(
    train_dataset_final,
    validation_data=val_dataset_small,
    epochs=5,
    callbacks=[early_stopping],
    verbose=1
)

# Predict on validation dataset
preds = model.predict(val_dataset)

In [ ]:
# preds is a list of 6 elements: [battery, camera, performance, design, display, sentiment_probs]
battery_pred = preds[0].flatten()
camera_pred = preds[1].flatten()
performance_pred = preds[2].flatten()
design_pred = preds[3].flatten()
display_pred = preds[4].flatten()
sentiment_probs = preds[5]

# Convert sentiment probabilities to class labels
sentiment_pred = np.argmax(sentiment_probs, axis=1)


# Collect true labels from val_dataset
y_true_battery = []
y_true_camera = []
y_true_performance = []
y_true_design = []
y_true_display = []
y_true_sentiment = []

for batch in val_dataset:
    y_batch = batch[1]
    y_true_battery.extend(y_batch['battery'].numpy())
    y_true_camera.extend(y_batch['camera'].numpy())
    y_true_performance.extend(y_batch['performance'].numpy())
    y_true_design.extend(y_batch['design'].numpy())
    y_true_display.extend(y_batch['display'].numpy())
    y_true_sentiment.extend(y_batch['sentiment'].numpy())

y_true_battery = np.array(y_true_battery)
y_true_camera = np.array(y_true_camera)
y_true_performance = np.array(y_true_performance)
y_true_design = np.array(y_true_design)
y_true_display = np.array(y_true_display)
y_true_sentiment = np.array(y_true_sentiment)


In [ ]:
# Evaluation
def evaluate_regression(y_true, y_pred, name=""):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{name} - MAE: {mae:.3f}, MSE: {mse:.3f}")


print("REGRESSION RESULTS (Aspect Ratings)")
evaluate_regression(y_true_battery, battery_pred, "Battery")
evaluate_regression(y_true_camera, camera_pred, "Camera")
evaluate_regression(y_true_performance, performance_pred, "Performance")
evaluate_regression(y_true_design, design_pred, "Design")
evaluate_regression(y_true_display, display_pred, "Display")


# Classification report
print("CLASSIFICATION RESULTS (Sentiment)")
print("\nSentiment Classification Report:")
print(classification_report(y_true_sentiment, sentiment_pred, target_names=["Negative","Neutral","Positive"], zero_division=0))

# Confusion matrix
cm = confusion_matrix(y_true_sentiment, sentiment_pred)
print("\nConfusion Matrix:")
print(cm)

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(
    cm, annot=True, fmt="d", cmap="Blues",
    xticklabels=["Negative","Neutral","Positive"],
    yticklabels=["Negative","Neutral","Positive"]
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Sentiment Confusion Matrix")
plt.show()


# TRAINING HISTORY (All Outputs)

output_heads = ["battery", "camera", "performance", "design", "display", "sentiment"]

fig, axes = plt.subplots(2, len(output_heads), figsize=(4*len(output_heads), 8))
fig.suptitle("Training History (All Outputs)", fontsize=16)

# Plot individual losses
for i, head in enumerate(output_heads):
    ax = axes[0, i]
    train_loss_key = f"{head}_loss"
    val_loss_key = f"val_{head}_loss"
    if train_loss_key in history.history:
        ax.plot(history.history[train_loss_key], label="Train")
        ax.plot(history.history[val_loss_key], label="Val")
        ax.set_title(f"{head.capitalize()} Loss")
        ax.legend()
    else:
        ax.axis("off")

# Plot metrics
for i, head in enumerate(output_heads):
    ax = axes[1, i]
    # For regression heads → MAE, for sentiment → accuracy
    metric_name = "mae" if head != "sentiment" else "accuracy"
    train_metric_key = f"{head}_{metric_name}"
    val_metric_key = f"val_{head}_{metric_name}"
    if train_metric_key in history.history:
        ax.plot(history.history[train_metric_key], label="Train")
        ax.plot(history.history[val_metric_key], label="Val")
        ax.set_title(f"{head.capitalize()} {metric_name.upper()}")
        ax.legend()
    else:
        ax.axis("off")

plt.tight_layout()
plt.show()


# TOTAL LOSS PLOT

plt.figure(figsize=(6,4))
plt.plot(history.history['loss'], label='Train Total Loss')
plt.plot(history.history['val_loss'], label='Val Total Loss')
plt.title("Overall Model Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Save the model and tokenizer
model.save("/kaggle/working/multitask_bert_model.keras") 
tokenizer.save_pretrained("/kaggle/working/multitask_bert_model")